In [27]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, AgentType, create_openai_functions_agent
from langchain.prompts import PromptTemplate

from langchain.agents import AgentExecutor

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [4]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki_tool.name

'wikipedia'

In [5]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

'arxiv'

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [7]:
embeddings = OllamaEmbeddings(model='mxbai-embed-large')
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001577D058340>, search_kwargs={})

In [8]:
retrieval_tool = create_retriever_tool(retriever=retriever, name="langsmith_search",
                                       description="Search for information about LangSmith. For any qeustion about langsmith you must use this tool.")
retrieval_tool.name

'langsmith_search'

In [18]:
for tool in tools:
    print(tool.name, tool.args)


wikipedia {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
arxiv {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}
langsmith_search {'query': {'description': 'query to look up in retriever', 'title': 'Query', 'type': 'string'}}


In [39]:

tools = [wiki_tool, arxiv_tool, retrieval_tool]
llm = OllamaLLM(model="llama3.2:1b")

prompt = PromptTemplate(
    input_variables=["input", "page_content"],
    template="Given the context: {page_content}, answer the question: {input}"
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt=prompt,  
    verbose=True  
)

response = agent.invoke({
    "input": "Tell me about Langsmith",
    "page_content": "LangSmith is a tool for managing AI workflows..."
})

print(response)

# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     prompt=prompt,  
#     verbose=True  
# )




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {''}

In [40]:
agent.input_keys

['', 'input', 'page_content']